In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score
import pickle


def get_metrics(model, axis, preprocessed_test=None):
    prob_preds = model.predict_proba(x_train)
    performance = roc_auc_score(y_train, prob_preds[:, axis])
    print "TRAINING: " + str(performance)
    
    prob_preds = model.predict_proba(x_test)
    performance = roc_auc_score(y_test, prob_preds[:, axis])
    print "TEST: " + str(performance)

In [2]:
x = np.load("data/x_normalized.npy")
y = np.load("data/y.npy")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
params = {'num_leaves': [6, 10, 15, 20, 25, 30, 35, 40, 45, 50], 
             'min_child_samples': [5, 10, 15, 20, 25, 30, 35, 40], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#              'subsample': sp_uniform(loc=0.2, scale=0.8), 
#              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
             'max_depth':[1, 3, 5, 8, 10, 15, 20],
             'n_estimators':[100, 500, 1000, 5000],
              'learning_rate':[0.01, 0.05, 0.1, 0.3, 0.5, 0.8]
            }

In [16]:
import lightgbm as lgb

train_data=lgb.Dataset(x_train,label=y_train)
best_validation_performance = 0.0
best_training_performance = 0.0
best_training_params = {}
best_validation_params = {}

for num_leaves in params['num_leaves']:
    for min_child_samples in params['min_child_samples']:
        for min_child_weight in params['min_child_weight']:
            for reg_alpha in params['reg_alpha']:
                for reg_lambda in params['reg_lambda']:
                    for max_depth in params['max_depth']:
                        for n_estimators in params['n_estimators']:
                            for learning_rate in params['learning_rate']:
                                temp_params =  {'learning_rate': learning_rate, 'num_leaves': num_leaves, 'reg_alpha': reg_alpha, 'min_child_samples': min_child_samples, 'min_child_weight': min_child_weight, 'n_estimators': n_estimators, 'reg_lambda': reg_lambda, 'max_depth': max_depth}
                                model = lgb.LGBMClassifier(learning_rate=learning_rate, num_leaves=num_leaves, reg_alpha=reg_alpha, min_child_samples=min_child_samples,min_child_weight=min_child_weight,n_estimators=n_estimators, reg_lambda=reg_lambda, max_depth=max_depth)
                                model.fit(x_train, y_train, eval_set=(x_test, y_test), eval_metric='auc', early_stopping_rounds=50, verbose=False)
                                
                                y_pred = model.predict_proba(x_train)[:, 1]
                                performance = roc_auc_score(y_train, y_pred)
                                
                                print "------------------------------"
                    
                                print "TRAINING: " + str(performance)
                                
                                if performance > best_training_performance:
                                    best_training_performance = performance
                                    best_training_params = temp_params
            
                                y_pred=model.predict_proba(x_test)[:, 1]
                                performance = roc_auc_score(y_test, y_pred)
                    
                                if performance > best_validation_performance:
                                    best_validation_performance = performance
                                    best_validation_params = temp_params
                                
                                print "TEST " + str(performance)
                                print temp_params
                                print "------------------------------"
                                

                                

------------------------------
TRAINING: 0.7250343442769476
TEST 0.7230497291995631
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 1}
------------------------------
------------------------------
TRAINING: 0.7523650226260109
TEST 0.7520472649285552
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 1}
------------------------------
------------------------------
TRAINING: 0.7638290732206592
TEST 0.764033558860521
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 1}
------------------------------
------------------------------
TRAINING: 0.7779074610129422
TEST 0.7768326462893139
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate'

------------------------------
TRAINING: 0.7795342933716845
TEST 0.7775750963285057
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 3}
------------------------------
------------------------------
TRAINING: 0.8040851771678483
TEST 0.7890910328403947
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 3}
------------------------------
------------------------------
TRAINING: 0.810170892602426
TEST 0.7898837795455411
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 3}
------------------------------
------------------------------
TRAINING: 0.8196502883012062
TEST 0.7886781595563994
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate'

------------------------------
TRAINING: 0.7908653495637785
TEST 0.7857476248614342
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 5}
------------------------------
------------------------------
TRAINING: 0.8205853008387697
TEST 0.7906939011784732
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 5}
------------------------------
------------------------------
TRAINING: 0.8269031026731231
TEST 0.7908274006236161
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 5}
------------------------------
------------------------------
TRAINING: 0.820046336521055
TEST 0.7900127790093873
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_r

------------------------------
TRAINING: 0.8136274689765883
TEST 0.7904003398985874
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 8}
------------------------------
------------------------------
TRAINING: 0.8205853008387697
TEST 0.7906939011784732
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 8}
------------------------------
------------------------------
TRAINING: 0.8269031026731231
TEST 0.7908274006236161
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 8}
------------------------------
------------------------------
TRAINING: 0.820046336521055
TEST 0.7900127790093873
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_r

------------------------------
TRAINING: 0.751102405233435
TEST 0.7485351076509588
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 15}
------------------------------
------------------------------
TRAINING: 0.7808535128356532
TEST 0.7787903100277739
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 15}
------------------------------
------------------------------
TRAINING: 0.7908178486111521
TEST 0.7855215945508728
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 15}
------------------------------
------------------------------
TRAINING: 0.8086191508688856
TEST 0.7891435951219327
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 0, 'learning_ra

...

------------------------------
TRAINING: 0.7908000802756066
TEST 0.785751031097277
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 20}
------------------------------
------------------------------
TRAINING: 0.8185163499050322
TEST 0.7909865249622556
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 20}
------------------------------
------------------------------
TRAINING: 0.8147795722219152
TEST 0.7904069648710522
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 20}
------------------------------
------------------------------
TRAINING: 0.8146955142889905
TEST 0.7893143444122561
{'n_estimators': 1000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0, 

------------------------------
TRAINING: 0.7814805897576911
TEST 0.7798352431847705
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 1}
------------------------------
------------------------------
TRAINING: 0.7910070702173962
TEST 0.7853350172013347
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 1}
------------------------------
------------------------------
TRAINING: 0.7902201432131584
TEST 0.7851080181447996
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 1}
------------------------------
------------------------------
TRAINING: 0.7922481597744475
TEST 0.785812327717513
{'n_estimators': 5000, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 

------------------------------
TRAINING: 0.7510917141246227
TEST 0.7485360763969324
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 5}
------------------------------
------------------------------
TRAINING: 0.7811154804221239
TEST 0.7792146988889077
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 5}
------------------------------
------------------------------
TRAINING: 0.7903301044059488
TEST 0.7852977829810895
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 5}
------------------------------
------------------------------
TRAINING: 0.8076526533799149
TEST 0.7885059415221806
{'n_estimators': 100, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1

------------------------------
TRAINING: 0.7808636893622909
TEST 0.778987559207957
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.01, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 8}
------------------------------
------------------------------
TRAINING: 0.8057654877004607
TEST 0.7895411872194407
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.05, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 8}
------------------------------
------------------------------
TRAINING: 0.8156419529122009
TEST 0.7903845899640478
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'learning_rate': 0.1, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 8}
------------------------------
------------------------------
TRAINING: 0.8139472439177479
TEST 0.7891950324081467
{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0.1, 'reg_lambda': 0.1,

KeyboardInterrupt: 

In [18]:
print best_validation_params
print best_validation_performance

{'n_estimators': 500, 'num_leaves': 6, 'reg_alpha': 0, 'reg_lambda': 1, 'learning_rate': 0.3, 'min_child_samples': 5, 'min_child_weight': 1e-05, 'max_depth': 3}
0.8326426932139277


In [48]:
x = np.load("data/x_normalized.npy")
y = np.load("data/y.npy")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(learning_rate=0.09, num_leaves=100, reg_alpha=0, min_child_samples=5, min_child_weight=1e-05, n_estimators=5000, reg_lambda=100, max_depth=10)
model.fit(x_train, y_train, eval_set=(x_test, y_test), eval_metric='auc', early_stopping_rounds=50, verbose=False)

y_pred = model.predict_proba(x_train)[:, 1]
performance = roc_auc_score(y_train, y_pred)
print performance

y_pred=model.predict_proba(x_test)[:, 1]
performance = roc_auc_score(y_test, y_pred)
print performance



0.8853723064609313
0.797676559656799
